In [19]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import Image
import gym
from gym import spaces

In [62]:
class CustomEnv(gym.Env):  #creating a custom class for environment
    metadata={'render.modes':['human']}
    
    def __init__(self):  
        super(CustomEnv,self).__init__()
        
        self.x_cell_size=np.random.randint(0,10)
        self.y_cell_size=np.random.randint(0,10)
        
        #initializing 4 discrete actions
        #4 discrete action 0-LEFT,1-RIGHT,2-UP,3-DOWN
        self.action_space=spaces.Discrete(4)
        
        #initializing observation spaces
        self.observation_space=spaces.Box(low=0,high=255,shape=(10,10,3),dtype=np.uint8)

#creating class for horizontal shape rectangle
class Horiz_Rectangle(CustomEnv):
    
    def __init__(self):
        super(CustomEnv,self).__init__()
        
        #coordinates for the block green
        z=np.random.randint(0,9)
        self.x_cell_size=([z,z+1])
        self.y_cell_size=np.random.randint(0,10)
        
        #coordinates check for out of bound
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 0:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0] > 9:
            self.x_cell_size[0] = 8
            
        if self.y_cell_size < 0:
            self.y_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9

        #initializing state and centroid of the object   
        self.state=[[self.x_cell_size[0],self.y_cell_size],[self.x_cell_size[1],self.y_cell_size]]
        self.centroid_x_cell_size=(((2*z)+1)/2) + 0.5
        self.centroid_y_cell_size=self.y_cell_size+0.5
        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
    
    #function to check the overlapping of machines
    def compare_elements(self,other):
        for x in range(len(self.state)):
            for y in range(len(other.state)):
                if self.state[x] == other.state[y]:
                    z=np.random.randint(0,9)
                    self.x_cell_size=([z,z+1]) #coordinates for the block green
                    self.y_cell_size=np.random.randint(0,10)
                    self.state=[[self.x_cell_size[0],self.y_cell_size],[self.x_cell_size[1],self.y_cell_size]]
                    self.centroid_x_cell_size=(((2*z)+1)/2) + 0.5
                    self.centroid_y_cell_size=self.y_cell_size+0.5
                    self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size     
                else:
                    self.state=self.state
        return self.state
    
    #function to move the machine and compare with other machine based on action
    def step(self,action,other):
        if action==0: #move left
            self.x_cell_size[0]=self.x_cell_size[0]-1
            self.x_cell_size[1]=self.x_cell_size[1]-1
        elif action==1: #move right
            self.x_cell_size[0]=self.x_cell_size[0]+1
            self.x_cell_size[1]=self.x_cell_size[1]+1
        elif action==2: #move up
            self.y_cell_size=self.y_cell_size-1
        elif action==3: #move down
            self.y_cell_size=self.y_cell_size+1
        
        #coordinates check for out of bound 
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 0:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0]> 9:
            self.x_cell_size[0]= 8
            
        if self.y_cell_size < 0:
            self.y_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9
        
        #initializing new state and new centroid of the object based on action
        self.state_new=self.state=[[self.x_cell_size[0],self.y_cell_size],[self.x_cell_size[1],self.y_cell_size]]
        z=self.x_cell_size[0]
        self.centroid_x_cell_size=(2*z+1)/2+0.5
        self.centroid_y_cell_size=self.y_cell_size+0.5
        self.centroid_state_new=self.centroid_x_cell_size,self.centroid_y_cell_size
        
        #initializing reward
        if self.centroid_x_cell_size-other.centroid_x_cell_size==0 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==0:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size==other.centroid_x_cell_size and self.centroid_y_cell_size==other.centroid_y_cell_size:
            reward=-1
            done=False
        else:
            reward=-1
            done=False
            
        info={}
        
        return self.state_new,(self.centroid_x_cell_size-other.centroid_x_cell_size,self.centroid_y_cell_size-other.centroid_y_cell_size),reward,done,info

#creating class for vertical shape rectangle
class Vert_Rectangle(CustomEnv):
    
    def __init__(self):
        super(CustomEnv,self).__init__()
        
        #coordinates for the block blue
        a=np.random.randint(1,10)
        self.y_cell_size=a
        self.y1_cell_size=a-1
        self.x_cell_size=np.random.randint(0,10)
        
        #coordinates check for out of bound
        if self.x_cell_size < 0:
            self.x_cell_size = 0
        elif self.x_cell_size> 9:
            self.x_cell_size= 9
            
        if self.y_cell_size< 1:
            self.y_cell_size= 1
        elif self.y1_cell_size < 0:
            self.y1_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9
        elif self.y1_cell_size> 8:
            self.y1_cell_size= 8
        
        #initializing state and centroid of the object
        self.state=[[self.x_cell_size,self.y1_cell_size],[self.x_cell_size,self.y_cell_size]]
        self.centroid_x_cell_size=self.x_cell_size+0.5
        self.centroid_y_cell_size=(2*a-1)/2+0.5
        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
            
    #function to check the overlapping of machines
    def compare_elements(self,other):
            for x in range(len(self.state)):
                for y in range(len(other.state)):
                    if self.state[x] == other.state[y]:
                        a=np.random.randint(1,10)
                        self.x_cell_size=np.random.randint(0,10)
                        self.y_cell_size=a
                        self.y1_cell_size=a-1
                        self.state=[[self.x_cell_size,self.y1_cell_size],[self.x_cell_size,self.y_cell_size]]
                        self.centroid_x_cell_size=self.x_cell_size+0.5
                        self.centroid_y_cell_size=(2*a-1)/2+0.5
                        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size      
                    else:
                        self.state=self.state
            return self.state
        
    #function to move the machine and compare with other machine based on action
    def step(self,action,other):     
        if action==0: #move left
            self.x_cell_size=self.x_cell_size-1
        elif action==1: #move right
            self.x_cell_size=self.x_cell_size+1
        elif action==2: #move up
            self.y_cell_size=self.y_cell_size-1
            self.y1_cell_size=self.y1_cell_size-1
        elif action==3: #move down
            self.y_cell_size=self.y_cell_size+1
            self.y1_cell_size=self.y1_cell_size+1
        
        #coordinates check for out of bound 
        if self.x_cell_size < 0:
            self.x_cell_size = 0
        elif self.x_cell_size> 9:
            self.x_cell_size= 9
            
        if self.y_cell_size< 1:
            self.y_cell_size= 1
        elif self.y1_cell_size < 0:
            self.y1_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9
        elif self.y1_cell_size> 8:
            self.y1_cell_size= 8

        #initializing new state and new centroid of the object based on action
        self.state_new=[[self.x_cell_size,self.y1_cell_size],[self.x_cell_size,self.y_cell_size]]
        a=self.y_cell_size
        self.centroid_x_cell_size=self.x_cell_size+0.5
        self.centroid_y_cell_size=(2*a-1)/2+0.5
        self.centroid_state_new=self.centroid_x_cell_size,self.centroid_y_cell_size
        
        #initializing reward
        if self.centroid_x_cell_size-other.centroid_x_cell_size==0 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==0:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size==other.centroid_x_cell_size and self.centroid_y_cell_size==other.centroid_y_cell_size:
            reward=-1
            done=False
        else:
            reward=-1
            done=False
            
        info={}
        
        return self.state_new,(self.centroid_x_cell_size-other.centroid_x_cell_size,self.centroid_y_cell_size-other.centroid_y_cell_size),reward,done,info

#creating class for small shape square
class Square(CustomEnv):
    
    def __init__(self):
        super(CustomEnv,self).__init__()
        
        #coordinates for the block red and yellow
        self.x_cell_size=np.random.randint(0,10)
        self.y_cell_size=np.random.randint(0,10)
        
        #coordinates check for out of bound
        if self.x_cell_size < 0:
            self.x_cell_size = 0
        elif self.x_cell_size> 9:
            self.x_cell_size= 9
        if self.y_cell_size< 0:
            self.y_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9
        
        #initializing state and centroid of the object
        self.state=[[self.x_cell_size,self.y_cell_size]]
        self.centroid_x_cell_size=self.x_cell_size+0.5
        self.centroid_y_cell_size=self.y_cell_size+0.5
        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
    
    #function to check the overlapping of machines
    def compare_elements(self,other):
                for x in range(len(self.state)):
                    for y in range(len(other.state)):
                        if self.state[x] == other.state[y]:
                            self.x_cell_size=np.random.randint(0,10)
                            self.y_cell_size=np.random.randint(0,10)
                            self.state=[[self.x_cell_size,self.y_cell_size]]
                            self.centroid_x_cell_size=self.x_cell_size+0.5
                            self.centroid_y_cell_size=self.y_cell_size+0.5
                            self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
                        else:
                            self.state=self.state
                return self.state
            
    #function to move the machine and compare with other machine based on action
    def step(self,action,other):
        if action==0: #move left
            self.x_cell_size=self.x_cell_size-1
        elif action==1: #move right
            self.x_cell_size=self.x_cell_size+1
        elif action==2: #move up
            self.y_cell_size=self.y_cell_size-1
        elif action==3: #move down
            self.y_cell_size=self.y_cell_size+1
        
        #coordinates check for out of bound 
        if self.x_cell_size < 0:
            self.x_cell_size = 0
        elif self.x_cell_size> 9:
            self.x_cell_size= 9
        if self.y_cell_size < 0:
            self.y_cell_size = 0
        elif self.y_cell_size> 9:
            self.y_cell_size= 9
        
        #initializing new state and new centroid of the object based on action
        self.state_new=[[self.x_cell_size,self.y_cell_size]]
        self.centroid_x_cell_size=self.x_cell_size+0.5
        self.centroid_y_cell_size=self.y_cell_size+0.5
        self.centroid_state_new=self.centroid_x_cell_size,self.centroid_y_cell_size
        
        #initializing reward
        if self.centroid_x_cell_size-other.centroid_x_cell_size==0 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==0:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size==other.centroid_x_cell_size and self.centroid_y_cell_size==other.centroid_y_cell_size:
            reward=-1
            done=False
        else:
            reward=-1
            done=False
            
        info={}
        
        return self.state_new,(self.centroid_x_cell_size-other.centroid_x_cell_size,self.centroid_y_cell_size-other.centroid_y_cell_size),self.centroid_state_new,reward,done,info

#creating class for L shape machine
class Lshape(CustomEnv):
    
    def __init__(self):
        super(CustomEnv,self).__init__()
        
        #coordinates for the block white
        z=np.random.randint(0,9)
        a=np.random.randint(1,10)
        self.x_cell_size=([z,z+1])
        self.y_cell_size=([a-1,a])
        
        #coordinates check for out of bound
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 0:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0]> 9:
            self.x_cell_size[0]= 8
        elif self.x_cell_size[1]>9:
            self.x_cell_size[1]= 9

        if self.y_cell_size[1]< 1:
            self.y_cell_size[1]= 1
        elif self.y_cell_size[0] < 0:
            self.y_cell_size[0] = 0
        elif self.y_cell_size[1]> 9:
            self.y_cell_size[1]= 9
        elif self.y_cell_size[0]> 8:
            self.y_cell_size[0]= 8

        #initializing state and centroid of the object    
        self.centroid_x_cell_size=self.x_cell_size[0]+(1/3)+0.5
        self.centroid_y_cell_size=self.y_cell_size[1]-(1/3)+0.5
        self.state=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[1]]]
        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
    
    #function to check the overlapping of machines
    def compare_elements(self,other):
                for x in range(len(self.state)):
                    for y in range(len(other.state)):
                        if self.state[x] == other.state[y]:
                            z=np.random.randint(0,9)
                            a=np.random.randint(1,10)
                            self.x_cell_size=([z,z+1])
                            self.y_cell_size=([a-1,a])
                            self.state=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[1]]]
                            self.centroid_x_cell_size=self.x_cell_size[0]+(1/3)+0.5
                            self.centroid_y_cell_size=self.y_cell_size[1]-(1/3)+0.5
                            self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
                        else:
                            self.state=self.state
                return self.state
            
    #function to move the machine and compare with other machine based on action
    def step(self,action,other):
        if action==0: #move left
            self.x_cell_size[0]=self.x_cell_size[0]-1
            self.x_cell_size[1]=self.x_cell_size[1]-1
        elif action==1: #move right
            self.x_cell_size[0]=self.x_cell_size[0]+1
            self.x_cell_size[1]=self.x_cell_size[1]+1
        elif action==2: #move up
            self.y_cell_size[0]=self.y_cell_size[0]-1
            self.y_cell_size[1]=self.y_cell_size[1]-1
        elif action==3: #move down
            self.y_cell_size[0]=self.y_cell_size[0]+1
            self.y_cell_size[1]=self.y_cell_size[1]+1
        
        #coordinates check for out of bound 
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 0:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0]> 9:
            self.x_cell_size[0]= 8
        elif self.x_cell_size[1]>9:
            self.x_cell_size[1]= 9

        if self.y_cell_size[1]< 1:
            self.y_cell_size[1]= 1
        elif self.y_cell_size[0] < 0:
            self.y_cell_size[0] = 0
        elif self.y_cell_size[1]> 9:
            self.y_cell_size[1]= 9
        elif self.y_cell_size[0]> 8:
            self.y_cell_size[0]= 8

        #initializing new state and new centroid of the object based on action
        self.state_new=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[1]]]
        self.centroid_x_cell_size=self.x_cell_size[0]+(1/3)+0.5
        self.centroid_y_cell_size=self.y_cell_size[1]-(1/3)+0.5
        self.centroid_state_new=self.centroid_x_cell_size,self.centroid_y_cell_size
        
        #initializing reward
        if self.centroid_x_cell_size-other.centroid_x_cell_size==0 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==0:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size==other.centroid_x_cell_size and self.centroid_y_cell_size==other.centroid_y_cell_size:
            reward=-1
            done=False
        else:
            reward=-1
            done=False
            
        info={}
        
        return self.state_new,(self.centroid_x_cell_size-other.centroid_x_cell_size,self.centroid_y_cell_size-other.centroid_y_cell_size),self.centroid_state_new,reward,done,info

#creating class for Big shape square machine
class Big_square(CustomEnv):
    
    def __init__(self):
        super(CustomEnv,self).__init__()
        
        #coordinates for the block lime
        z=np.random.randint(0,9)
        a=np.random.randint(1,10)
        self.x_cell_size=([z,z+1])
        self.y_cell_size=([a-1,a])
        
        #coordinates check for out of bound
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 1:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0]> 9:
            self.x_cell_size[0]= 8
        elif self.x_cell_size[1]>9:
            self.x_cell_size[1]= 9

        if self.y_cell_size[1]< 1:
            self.y_cell_size[1]= 1
        elif self.y_cell_size[0] < 0:
            self.y_cell_size[0] = 0
        elif self.y_cell_size[1]> 9:
            self.y_cell_size[1]= 9
        elif self.y_cell_size[0]> 8:
            self.y_cell_size[0]= 8

        #initializing state and centroid of the object
        self.centroid_x_cell_size=self.x_cell_size[0]+1
        self.centroid_y_cell_size=self.y_cell_size[1]
        self.state=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[0]],[self.x_cell_size[1],self.y_cell_size[1]]]
        self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
    
    #function to check the overlapping of machines
    def compare_elements(self,other):
                for x in range(len(self.state)):
                    for y in range(len(other.state)):
                        if self.state[x] == other.state[y]:
                            z=np.random.randint(0,9)
                            a=np.random.randint(1,10)
                            self.x_cell_size=([z,z+1])
                            self.y_cell_size=([a-1,a])
                            self.state=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[0]],[self.x_cell_size[1],self.y_cell_size[1]]]
                            self.centroid_x_cell_size=self.x_cell_size[0]+1
                            self.centroid_y_cell_size=self.y_cell_size[1]
                            self.centroid_state=self.centroid_x_cell_size,self.centroid_y_cell_size
                        else:
                            self.state=self.state
                return self.state
    
    #function to move the machine and compare with other machine based on action
    def step(self,action,other):
        if action==0: #move left
            self.x_cell_size[0]=self.x_cell_size[0]-1
            self.x_cell_size[1]=self.x_cell_size[1]-1
        elif action==1: #move right
            self.x_cell_size[0]=self.x_cell_size[0]+1
            self.x_cell_size[1]=self.x_cell_size[1]+1
        elif action==2: #move up
            self.y_cell_size[0]=self.y_cell_size[0]-1
            self.y_cell_size[1]=self.y_cell_size[1]-1
        elif action==3: #move down
            self.y_cell_size[0]=self.y_cell_size[0]+1
            self.y_cell_size[1]=self.y_cell_size[1]+1
        
        #coordinates check for out of bound
        if self.x_cell_size[0] < 0:
            self.x_cell_size[0] = 0
        elif self.x_cell_size[1] < 1:
            self.x_cell_size[1] = 1
        elif self.x_cell_size[0]> 9:
            self.x_cell_size[0]= 8
        elif self.x_cell_size[1]>9:
            self.x_cell_size[1]= 9

        if self.y_cell_size[1]< 1:
            self.y_cell_size[1]= 1
        elif self.y_cell_size[0] < 0:
            self.y_cell_size[0] = 0
        elif self.y_cell_size[1]> 9:
            self.y_cell_size[1]= 9
        elif self.y_cell_size[0]> 8:
            self.y_cell_size[0]= 8
        
        #initializing new state and new centroid of the object based on action
        self.state_new=self.state=[[self.x_cell_size[0],self.y_cell_size[0]],[self.x_cell_size[0],self.y_cell_size[1]],[self.x_cell_size[1],self.y_cell_size[0]],[self.x_cell_size[1],self.y_cell_size[1]]]
        self.centroid_x_cell_size=self.x_cell_size[0]+1
        self.centroid_y_cell_size=self.y_cell_size[1]
        self.centroid_state_new=self.centroid_x_cell_size,self.centroid_y_cell_size
        
        #initializing reward
        if self.centroid_x_cell_size-other.centroid_x_cell_size==0 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==0:
            reward=1
            done=True
        elif self.centroid_x_cell_size-other.centroid_x_cell_size==1|-1 and self.centroid_y_cell_size-other.centroid_y_cell_size==1|-1:
            reward=1
            done=True
        elif self.centroid_x_cell_size==other.centroid_x_cell_size and self.centroid_y_cell_size==other.centroid_y_cell_size:
            reward=-1
            done=False
        else:
            reward=-1
            done=False
            
        info={}
        
        return self.state_new,(self.centroid_x_cell_size-other.centroid_x_cell_size,self.centroid_y_cell_size-other.centroid_y_cell_size),self.centroid_state_new,reward,done,info

    def reset(self):
        self.state=self.x_cell_size,self.y_cell_size
        return self.state
    def render(self,mode='human'):
        pass
    def close(self):
        pass
 
output_window=np.zeros((10,10,3),dtype='uint8')
object1=Horiz_Rectangle()
object2=Vert_Rectangle()
object3=Square()
object4=Square()
object5=Lshape()
object6=Big_square()

output_window[object1.y_cell_size][object1.x_cell_size]=0,255,0
output_window[object2.y_cell_size][object2.x_cell_size]=255,0,0
output_window[object2.y1_cell_size][object2.x_cell_size]=255,0,0
output_window[object3.y_cell_size][object3.x_cell_size]=0,0,255
output_window[object4.y_cell_size][object4.x_cell_size]=0,255,255
output_window[object5.y_cell_size[1]][object5.x_cell_size]=255,255,255
output_window[object5.y_cell_size[0]][object5.x_cell_size[0]]=255,255,255
output_window[object6.y_cell_size[1]][object6.x_cell_size]=255,255,0
output_window[object6.y_cell_size[0]][object6.x_cell_size]=255,255,0

img=Image.fromarray(output_window)
output_img=img.resize((500,500),resample=Image.BOX)
cv.imshow('output_window',np.array(output_img))
cv.waitKey(0)

print(object1.state)
print(object2.state)
print(object3.state)
print(object4.state)
print(object5.state)
print(object6.state)
print(object1.compare_elements(object2))
print(object1.compare_elements(object3))
print(object1.compare_elements(object4))
print(object1.compare_elements(object5))
print(object1.compare_elements(object6))
print(object2.compare_elements(object3))
print(object2.compare_elements(object4))                                           
print(object2.compare_elements(object5))
print(object2.compare_elements(object6))  
print(object3.compare_elements(object4))
print(object3.compare_elements(object5))
print(object3.compare_elements(object6)) 
print(object4.compare_elements(object5)) 
print(object4.compare_elements(object6))
print(object5.compare_elements(object6))                                           
print(object1.centroid_state)
print(object2.centroid_state)
print(object3.centroid_state)
print(object4.centroid_state)
print(object5.centroid_state)
print(object6.centroid_state)
print(object1.step(1,object2))
print(object1.centroid_state_new)

[[5, 7], [6, 7]]
[[3, 7], [3, 8]]
[[8, 7]]
[[7, 9]]
[[0, 6], [0, 7], [1, 7]]
[[2, 4], [2, 5], [3, 4], [3, 5]]
[[5, 7], [6, 7]]
[[5, 7], [6, 7]]
[[5, 7], [6, 7]]
[[5, 7], [6, 7]]
[[5, 7], [6, 7]]
[[3, 7], [3, 8]]
[[3, 7], [3, 8]]
[[3, 7], [3, 8]]
[[3, 7], [3, 8]]
[[8, 7]]
[[8, 7]]
[[8, 7]]
[[7, 9]]
[[7, 9]]
[[0, 6], [0, 7], [1, 7]]
(6.0, 7.5)
(3.5, 8.0)
(8.5, 7.5)
(7.5, 9.5)
(0.8333333333333333, 7.166666666666667)
(3, 5)
([[6, 7], [7, 7]], (3.5, -0.5), -1, False, {})
(7.0, 7.5)
